In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf

#learning
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# different models
from tensorflow.keras.applications import efficientnet_v2

# optimization
from tensorflow_addons.optimizers import AdamW

#data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#preprocessing images
import cv2

## Preprocessing and loading in the data

In [11]:
# the values per image that belong to the given attributes for each class
attribute_values = np.load('feather-in-focus/attributes.npy')

# names of each bird class
labels = np.load("feather-in-focus/class_names.npy", allow_pickle=True).item()

# list of training data containing: image location, labels per image 
data = pd.read_csv("feather-in-focus/train_images.csv")


In [12]:
# code that adds the attributes per label to the training_df
att_list = []
for i in range(len(data)):
    att_list.append(attribute_values[data['label'][i]-1])  
    
data['attributes'] = att_list

In [13]:
def load_and_preprocess_images(image_paths, file_path):
    images = []
    for img_path in image_paths:
        img = cv2.imread(f'feather-in-focus/{file_path}{img_path}')
        img = cv2.resize(img, (350, 350))  # Resize the image

        # Normalize pixel values using EfficientNetV2's preprocess_input
        img = efficientnet_v2.preprocess_input(img)

        images.append(img)
    return np.array(images)

In [14]:
# Load and preprocess images
X_images = load_and_preprocess_images(data['image_path'], 'train_images')
y_labels = to_categorical(data['label'].values-1)

X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_size=0.2, random_state=0)

## Building the model


In [16]:
# Create Xception model with pre-trained weights
base_model =  efficientnet_v2.EfficientNetV2B3(include_top=False, weights='imagenet', input_shape=(350, 350, 3), classes=200)
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for classification
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5)) #0.5 works best
model.add(layers.Dense(y_train.shape[1], activation='softmax'))


model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])


# allow model to quit when not seeing improvement
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=3,
    min_delta=0.001,
    restore_best_weights=True,)


# Train the model
epochs = 20  
model.fit(np.array(X_train), y_train, epochs=epochs, validation_data=(np.array(X_test), y_test), callbacks=[early_stop])
# model.fit(augmented_data, epochs=epochs, validation_data=(np.array(X_test), y_test)) <- use this if you want to use the augmented data, didnt do much when used


# Evaluate the model
test_loss, test_acc = model.evaluate(np.array(X_test), y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/20
99/99 [==============================] - 109s 1s/step - loss: 4.7073 - accuracy: 0.0844 - val_loss: 3.7641 - val_accuracy: 0.1768
Epoch 2/20
99/99 [==============================] - 100s 1s/step - loss: 3.4328 - accuracy: 0.2223 - val_loss: 2.9233 - val_accuracy: 0.3346
Epoch 3/20
99/99 [==============================] - 99s 1s/step - loss: 2.7811 - accuracy: 0.3232 - val_loss: 2.4534 - val_accuracy: 0.3804
Epoch 4/20
99/99 [==============================] - 105s 1s/step - loss: 2.3963 - accuracy: 0.3748 - val_loss: 2.2170 - val_accuracy: 0.4186
Epoch 5/20
99/99 [==============================] - 99s 999ms/step - loss: 2.0819 - accuracy: 0.4513 - val_loss: 2.0244 - val_accuracy: 0.4631
Epoch 6/20
99/99 [==============================] - 93s 938ms/step - loss: 1.8539 - accuracy: 0.4946 - val_loss: 1.9455 - val_accuracy: 0.4669
Epoch 7/20
99/99 [==============================] - 88s 891ms/step - loss: 1.7021 - accuracy: 0.5366 - val_loss: 1.8036 - val_accuracy: 0.5051
Epoch 8

In [17]:
# Unfreeze a few layers at a time
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model after unfreezing layers
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
model.fit(np.array(X_train), y_train, epochs=5, validation_data=(np.array(X_test), y_test)) 


Epoch 1/5
99/99 [==============================] - 118s 1s/step - loss: 1.1408 - accuracy: 0.6682 - val_loss: 1.5896 - val_accuracy: 0.5509
Epoch 2/5
99/99 [==============================] - 105s 1s/step - loss: 0.9095 - accuracy: 0.7283 - val_loss: 1.5087 - val_accuracy: 0.5776
Epoch 3/5
99/99 [==============================] - 106s 1s/step - loss: 0.7837 - accuracy: 0.7637 - val_loss: 1.5342 - val_accuracy: 0.5878
Epoch 4/5
99/99 [==============================] - 102s 1s/step - loss: 0.7154 - accuracy: 0.7793 - val_loss: 1.5201 - val_accuracy: 0.5941
Epoch 5/5
99/99 [==============================] - 103s 1s/step - loss: 0.6438 - accuracy: 0.8083 - val_loss: 1.4687 - val_accuracy: 0.6018


## predicting the submission images and saving them 


In [18]:

test_df = pd.read_csv("feather-in-focus/test_images_path.csv")
test_images = load_and_preprocess_images(test_df['image_path'],'test_images')

# Predict
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)


submission_df = pd.DataFrame({
    'id': test_df['id'],
    'label': predicted_labels+1
})
submission_df.to_csv('submission.csv', index=False)

125/125 [==============================] - 96s 750ms/step
